# Asignement 3 = Stag Hunt model 
By Māra Učelniece 

- Response : there is not a specific data set we have to use, but just apply the evaluation of the structures to the EV's case that we are attempting to create policy interventions for. 

## Questions : 
- Ask Noah : Do we want to find a data set ? 

### Task : 
Evaluate how varying structures relate to policy and in which cases they might apply. 
   

In [12]:
# Imports 
    # Copied from exmaple notebook 
    
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
import networkx as nx
import numpy as np
import pandas as pd
import csv
import random
from typing import Iterable, List, Dict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed


# Can import the functions from the course_matterial 
    # Example : from file import function
from course_matterial.ev_core import *     
    # Example to import all functions
        # from ./course_matterial/ev_core.py import *
print("Done")

Done


## 0. Implement a network agent based model

In [13]:
# Generate real-world network
def build_real_network():
    # Load CSV network
    df = pd.read_csv('data/bowdoin_network.csv')
    # Expect columns 'Source' and 'Target' as in the provided CSV
    G = nx.from_pandas_edgelist(df, source='source', target='target', create_using=nx.Graph())
    # Remove self-loops if present
    G.remove_edges_from(list(nx.selfloop_edges(G)))
    # Remove components that are not the biggest connected component
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()
    print(f"Loaded empirical graph from 3_assignment_NM/data/bowdoin_network.csv: n={G.number_of_nodes()}, m={G.number_of_edges()}")
    return {'Bowdoin47': G}

# Generate model networks Erdos-Renyi, Watts-Strogatz and Barabasi-Albert with the same number of nodes and average degree as the real-world network that we can call on later in the notebook to conduct baselines system analysis and network structure analysis. Assign these model networks to objects with the keys being the model names appended to the real-world network name.
def generate_model_networks(real_graphs: Dict[str, nx.Graph]) -> Dict[str, nx.Graph]:
    model_graphs = {}
    for name, G in real_graphs.items():
        n = G.number_of_nodes()
        m = G.number_of_edges()
        avg_degree = 2 * m / n

        # Erdos-Renyi
        p = avg_degree / (n - 1)
        er_graph = nx.erdos_renyi_graph(n, p)
        model_graphs[f"{name}_ErdosRenyi"] = er_graph

        # Watts-Strogatz
        k = int(avg_degree)
        if k % 2 != 0:
            k += 1  # Ensure k is even
        ws_graph = nx.watts_strogatz_graph(n, k, 0.1)
        model_graphs[f"{name}_WattsStrogatz"] = ws_graph

        # Barabasi-Albert
        m_ba = max(1, int(avg_degree / 2))
        ba_graph = nx.barabasi_albert_graph(n, m_ba)
        model_graphs[f"{name}_BarabasiAlbert"] = ba_graph

    return model_graphs

# Call the function to generate the real-world network
graphs = build_real_network()
{name: (G.number_of_nodes(), G.number_of_edges()) for name, G in graphs.items()}

# Call the function to generate model networks
model_graphs = generate_model_networks(graphs)
all_graphs = {**graphs, **model_graphs}
{name: (G.number_of_nodes(), G.number_of_edges()) for name, G in all_graphs.items()}

bowdoin47_RW = graphs['Bowdoin47']
bowdoin47_ER = model_graphs['Bowdoin47_ErdosRenyi']   
bowdoin47_WS = model_graphs['Bowdoin47_WattsStrogatz']
bowdoin47_BA = model_graphs['Bowdoin47_BarabasiAlbert']

# Def advanced in ev_core =  applicable cuz then we can adjsut the strategy if want to evaluate the rule

# But tghe class EVStagHuntModel(Model) also ... 

# Only till set_intial_adaptors the script seems usefull for now 


Loaded empirical graph from 3_assignment_NM/data/bowdoin_network.csv: n=2250, m=84386


In [14]:
# Initialize the model with the pre-existing graph
    #  EVAgent(Agent) is in EVStagHuntModel(Model), so only need to call this 
    
model = EVStagHuntModel(
    G=bowdoin47_RW,
    initial_ev=10,  # Number of initial EV nodes
    a0=2.0,        # Base payoff for EV adoption
    beta_I=3.0,    # Payoff enhancement factor for EV adoption
    b=1.0,         # Payoff for ICE defection
    g_I=0.1,       # Infrastructure growth rate
    I0=0.05,       # Initial infrastructure level
    seed=42,       # Random seed for reproducibility
    collect=True,  # Whether to collect agent and model-level data
    strategy_choice_func="imitate",  # Strategy selection function
    tau=1.0        # Temperature parameter for softmax choice (only used with "logit")
)

AttributeError: 'EVStagHuntModel' object has no attribute 'G'

In [ ]:
# Step 3: Run the model for a specified number of steps
num_steps = 100
for i in range(num_steps):
    model.step()

# Step 4: Collect and analyze data
data = model.datacollector.get_model_vars_dataframe()
print(data)

# Optionally, you can also collect agent-level data
agent_data = model.datacollector.get_agent_vars_dataframe()
print(agent_data)

In [ ]:
# Set parameters for the trial
X0_frac = 0.1  # Initial fraction of EV adopters
ratio = 2.0    # Payoff ratio
I0 = 0.05      # Initial infrastructure level
beta_I = 2.0   # Payoff enhancement factor
b = 1.0        # Payoff for ICE defection
g_I = 0.05     # Infrastructure growth rate
T = 200        # Number of time steps
seed = 10      # Random seed for reproducibility
tol = 1e-3     # Tolerance for stability
patience = 30  # Patience for early stopping
collect = False # Whether to collect data
strategy_choice_func = "imitate" # Strategy selection function
tau = 1.0      # Temperature parameter for logit choice


# Run the trial
final_adoption_fraction = run_network_trial(
    G=bowdoin47_RW,
    X0_frac=X0_frac,
    ratio=ratio,
    I0=I0,
    beta_I=beta_I,
    b=b,
    g_I=g_I,
    T=T,
    seed=seed,
    tol=tol,
    patience=patience,
    collect=collect,
    strategy_choice_func=strategy_choice_func,
    tau=tau,
)

print(f"Final adoption fraction: {final_adoption_fraction}")

In [15]:
import inspect
print(inspect.getsource(run_network_trial))


def run_network_trial(
    G,
    X0_frac: float,
    ratio: float,
    *,
    I0: float = 0.05,
    beta_I: float = 2.0,
    b: float = 1.0,
    g_I: float = 0.05,
    T: int = 200,
    #network_type: str = "random",
    #n_nodes: int = 120,
    #p: float = 0.05,
    #m: int = 2,
    seed: int = 10, 
    tol: float = 1e-3,
    patience: int = 30,
    collect: bool = False,
    strategy_choice_func: str = "imitate",
    tau: float = 1.0,
) -> float:
        
    """Run a single realisation and return final adoption fraction.

    Preserves the intended initial payoff ratio via a0 = ratio*b - beta_I*I0.
    Includes basic stability-based early stopping.
    """
    initial_ev = int(round(X0_frac * G.number_of_nodes()))
    a0 = ratio * b - beta_I * I0

    model = EVStagHuntModel(
        self=self,
        G=G,
        initial_ev=initial_ev,
        a0=a0,
        beta_I=beta_I,
        b=b,
        g_I=g_I,
        I0=I0,
        seed=seed,
        #network_type=network_type,
        

## 1. Baseline System Analysis

In [ ]:
# Evaluate trough plots 




## 2. Network Structure Analysis

Seems that the follwoing will be done with plots 
- speedofadoption,
- probability of reaching the high-adoption equilibrium,
- cluster formation,
- network-specific sensitivity to tipping.

In [4]:
# Mainly could use ev_plotting for this, but there is also some matterial in ev_experiments


## 3. Policy Intervention Designand Evaluation

In [5]:
# For this one the ev_experiments mainly perevelent: 
    # policy_subsidy_factory = Create a policy that temporarily boosts coordination payoffs
    # policy_infrastructure_boost_factory = Create a policy that injects infrastructure at a specific step
    
# However, these are only two interventions, and we wnat to test possibly more or others 
    # Can still be used for insparation / a basis to start from 
    